In [1]:
import pandas as pd
# import surprise
from surprise import Reader, Dataset, SVD
from collections import defaultdict
import numpy as np

In [2]:
df = pd.read_csv("./data/netflix/data_fin.csv", sep=";")


In [3]:
# Посмотрим, сколько у нас всего уникальных фильмов:
df.Movie_Id.nunique()

17770

In [4]:
# Также мы можем посмотреть, сколько у нас представлено пользователей:
df.Cust_Id.nunique()

480189

In [5]:
# Для того, чтобы алгоритм работал быстрее, мы будем использовать только часть пользователей и фильмов.
# Сократим наши данные. Оставим только 10000 пользователей и 5000 фильмов:
cust_sample = df.Cust_Id.sample(10000)
movie_sample = df.Movie_Id.sample(5000)


In [6]:
# И подгружаем в surprise выбранный нами датасет:
# целый, семплированный потом посмотрим, если потребуется
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[["Cust_Id", "Movie_Id", "Rating"]], reader)


In [7]:
# Задаём наш алгоритм и формируем обучающую выборку:
svd = SVD()
trainingSet = data.build_full_trainset()

In [8]:
# Тренируем наш алгоритм:
svd.fit(trainingSet)

In [ ]:
# Далее мы уже можем получить предсказание:
testSet = trainingSet.build_testset()
predictions = svd.test(testSet)


In [ ]:
# Подгружаем названия фильмов:
titles = pd.read_csv(
    "./data/netflix/movie_titles.csv",
    encoding="ISO-8859-1",
    header=None,
    names=["Movie_Id", "Year", "Name"],
)

In [ ]:
def get_top_n_recommendations(predictions, top_n=3):

    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))

    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_recs[uid] = user_ratings[:top_n]

    return top_recs

In [ ]:
top3_recommendation = get_top_n_recommendations(predictions, top_n=3)

In [ ]:
def print_recs(user_id, recommendation):
    for (a, b) in recommendation[user_id]:
        print(titles[titles.Movie_id == a]["Name"].values[0], np.round(b, 2))

In [ ]:
user_id = 262149
print_recs(user_id, top3_recommendation)